In [ ]:
# Highway Car Count by YOLOv8 
https://docs.ultralytics.com/modes/track/<br/>
https://www.kaggle.com/code/stpeteishii/trafiic-tracking-by-yolov8


In [ ]:
!rm -rf yolov8_tracking

In [ ]:
!pip install ultralytics

In [ ]:
import os
import shutil
import cv2
import numpy as np
import pandas as pd
from IPython.display import Video
from ultralytics import YOLO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [ ]:
!mkdir yolov8_tracking
!mkdir yolov8_tracking/examples
!mkdir yolov8_tracking/examples/weights

In [ ]:
os.chdir('/kaggle/working/yolov8_tracking')
shutil.copy('/kaggle/input/my-movie/trim_british_highway_traffic.mp4', '/kaggle/working/yolov8_tracking/sample.mp4')
input_video_path='/kaggle/working/yolov8_tracking/sample.mp4'

In [ ]:
#Video(input_video_path, width=640, height=400, embed=True)

### !yolo mode=track tracker="bytetrack.yaml" model=yolov8n.pt conf=0.25 source={input_video_path} save=True save_txt=True

In [ ]:
!yolo mode=track tracker="bytetrack.yaml" model=yolov8n.pt conf=0.1 source={input_video_path} save=True save_txt=True

In [ ]:
# all successful
# !yolo mode=track model=yolov8n.pt source={input_video_path} 
# !yolo mode=track tracker="botsort.yaml" model=yolov8n.pt conf=0.25 source={input_video_path} save=True 
# !yolo mode=track tracker="bytetrack.yaml" model=yolov8n.pt conf=0.25 source={input_video_path} save=True save_txt=True

In [ ]:
output_video_path='/kaggle/working/yolov8_tracking/runs/detect/predict/sample.avi'
!ffmpeg -y -i {output_video_path} -vf "fps=10" -loop 0 detections.gif
from IPython.display import Image
Image(open('./detections.gif','rb').read())

# Analyse Result of Tracking

In [ ]:
txt_dir='/kaggle/working/yolov8_tracking/runs/detect/predict/labels'
txtpaths=[]
texts=os.listdir(txt_dir)
for item in texts:
    txtpaths+=[os.path.join(txt_dir,item)]

In [ ]:
boxdata=[]
boxfile=[]
for i in range(len(txtpaths)):
    file=txtpaths[i]
    boxdata+=[np.loadtxt(file)] 
    filename=file.split('/')[-1]
    if filename=='output.txt':
        filename='output_0.txt'
    boxfile+=[filename[0:-4]]
#print(boxdata[0:1])
#print(boxfile)

In [ ]:
BOX=pd.DataFrame(columns=range(7))

for i in range(len(boxdata)):
    if type(boxdata[i][0])==np.float64:
        add=pd.DataFrame([boxdata[i]])
        add[6]=boxfile[i]
        BOX=pd.concat([BOX,add],axis=0)
    else:
        add=pd.DataFrame(boxdata[i])
        add[6]=boxfile[i]
        #print(add)
        #print(add.shape)        
        BOX=pd.concat([BOX,add],axis=0)       

BOX2=BOX.iloc[:,0:7][BOX[0]==2]
BOX2[0:5]

In [ ]:
BOX2.columns=['class','x','y','w','h','id','file']
#print(BOX2['file'].unique().tolist())
BOX2['order']=BOX2['file'].apply(lambda x:x.split('_')[-1].zfill(3))
#print(BOX2['order'].unique().tolist())
data0=BOX2.sort_values('order',ascending=True).reset_index(drop=True)
display(data0)

In [ ]:
ids=sorted(data0['id'].unique().tolist())
orders=sorted(data0['order'].unique().tolist())
orders=pd.DataFrame(columns=['order'],data=orders)

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
for i in range(len(ids)):
    idi=ids[i]
    datai=data0[data0['id']==idi]
    datai=orders.merge(datai,on='order',how='left')
    idi2='id:'+str(int(idi))
    fig.add_trace(go.Scatter(x=datai['x'],y=-datai['y'],name=idi2,hovertext=idi2),secondary_y=False)
fig.update_layout(autosize=False,width=700,height=500,title_text="Traffic tracking")
fig.update_xaxes(title_text="x")
fig.update_yaxes(title_text="y",secondary_y=False)
fig.show()

# Draw Tracking Line

In [ ]:
!mkdir /kaggle/working/frame

In [ ]:
def video_2_frames(video_file=input_video_path, image_dir='/kaggle/working/frame/', image_file='img_%s.png'):
    i = 0
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag == False:
            break
        cv2.imwrite(image_dir+image_file % str(i).zfill(3), frame) 
        i += 1
    cap.release()

In [ ]:
video_2_frames()

In [ ]:
paths=[]
for dirname, _, filenames in os.walk('/kaggle/working/frame'):
    for filename in filenames:
        paths+=[(os.path.join(dirname, filename))]
paths=sorted(paths)
print(paths[0:3])

In [ ]:
data0

* cv2.rectangle(image, pt1, pt2, color, thickness, lineType, shift)
* cv2.putText(image, text, org, fontFace, fontScale, color, thickness, lineType, bottomLeftOrigin)


In [ ]:
images2=[]
for i,path in enumerate(paths):
    
    order=path.split('/')[-1][4:7]
    img = cv2.imread(path)
    img = cv2.resize(img,dsize=None,fx=0.3,fy=0.3)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

    H,W=img.shape[0],img.shape[1]
    cv2.rectangle(img,(0,30),(W,90),(255,0,0),2)
    
    datai=data0[data0['order']==order]
    x,y=datai.iloc[:,1],datai.iloc[:,2]
    datai2=datai[(y*H>=30)&(y*H<=90)]
    
    t=len(datai2)
    
    cv2.putText(img,str(t),(W//2,60), 
            cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)
    for j in range(len(datai2)):
        xj,yj=datai2.iloc[j,1],datai2.iloc[j,2]
        cv2.circle(img,(int(xj*W),int(yj*H)),2,(255,0,0),-1) 
    images2+=[img]
 

In [ ]:
import imageio

def create_animation_gif2(images, output_path, duration=100):
    imageio.mimsave(output_path, images, format='GIF', duration=duration/100)
    print(f"Animation GIF saved at {output_path}")    
    
output_path = 'output.gif'
create_animation_gif2(images2, output_path)

from IPython.display import Image
Image(open(output_path, 'rb').read())

    paths2=[]
    for dirname, _, filenames in os.walk('/kaggle/working/frame2'):
        for filename in filenames:
            paths2+=[(os.path.join(dirname, filename))]
    paths2=sorted(paths2)
    print(len(paths2))

    import imageio

    def create_animation_gif(image_paths, output_path, duration=100):
        images = []
        for image_path in image_paths:
            images.append(imageio.imread(image_path))

        imageio.mimsave(output_path, images, format='GIF', duration=duration/100)
        print(f"Animation GIF saved at {output_path}")    

    output_path = 'output.gif'
    create_animation_gif(paths2, output_path)

    from IPython.display import Image
    Image(open(output_path, 'rb').read())